In [1]:
from dotenv import load_dotenv
import openai
import os
import base64 

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)

In [ ]:
prompt ="""
    You are a helpful assistant that extracts information in JSON format.
    Do not add any additional information - only the JSON.
    Add only the information in the invoice and nothing else. 
    Do not add any additional information only the information in the invoice.
    Extract the following information from the image and return it in JSON format:
    {
  "invoiceNumber": "INV-1001",
  "date": "2024-12-10",
  "billingFrom": {
    "name": "Your Company Name",
    "address": "123 Business St, City, Country",
    "email": "yourcompany@example.com",
    "phone": "+1234567890"
  },
  "billingTo": {
    "name": "Client Name",
    "address": "456 Client Rd, City, Country",
    "email": "client@example.com",
    "phone": "+0987654321"
  },
  "items": [
    {
      "description": "Service or Product Description",
      "quantity": 2,
      "unitPrice": 150.00,
      "total": 300.00
    },
    {
      "description": "Service or Product Description",
      "quantity": 1,
      "unitPrice": 200.00,
      "total": 200.00
    }
  ],
  "subtotal": 500.00,
  "tax": 50.00,
  "total": 550.00,
}
"""
    
def call_gpt4o_with_imgs(image_list):
    text=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": [
        *map(lambda x: {"type": "image_url", 
                        "image_url": {
                            "url": f'data:image/jpg;base64,{x}', 
                            "detail": "low"}}, image_list)
        ],
    }
    ]
    response = client.chat.completions.create(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
        messages = text,
        temperature=0.0
    )
    return response.choices[0].message.content

In [6]:
import sys
from pdf2image import convert_from_bytes

base64Frames = []
invoicesPath = "./invoices/pdf/"
for filename in os.listdir(invoicesPath):
    idx = 0
    #transform pdf to image
    images = convert_from_bytes(open(invoicesPath+filename, "rb").read())
    for i in range(len(images)):
        #save images
        images[i].save(f"{invoicesPath}{filename}-page-{i}.jpg")
        with open(f"{invoicesPath}{filename}-page-{i}.jpg", "rb") as image_file:
            img64 = base64.b64encode(image_file.read()).decode("utf-8")
            #encode images to base64
            base64Frames.append(img64)
    result = call_gpt4o_with_imgs(base64Frames)
    print(result)



```json
{
  "invoiceNumber": "123/00081",
  "date": "01.03.2024",
  "billingFrom": {
    "name": "CPB SOFTWARE (GERMANY) GMBH",
    "address": "Im Bruch 3, 53567 Asbach",
    "email": "info@cpb-software.com",
    "phone": "+49 2683 9391 0"
  },
  "billingTo": {
    "name": "Musterkunde AG",
    "address": "Musterstr. 23, 12345 Musterstadt",
    "email": "N/A",
    "phone": "N/A"
  },
  "items": [
    {
      "description": "Basic Fee",
      "quantity": 1,
      "unitPrice": 100.00,
      "total": 100.00
    },
    {
      "description": "Data Access Fee",
      "quantity": 1,
      "unitPrice": 200.00,
      "total": 200.00
    },
    {
      "description": "Service Fee",
      "quantity": 1,
      "unitPrice": 150.00,
      "total": 150.00
    },
    {
      "description": "Additional Service",
      "quantity": 1,
      "unitPrice": 50.00,
      "total": 50.00
    }
  ],
  "subtotal": 500.00,
  "tax": 95.00,
  "total": 595.00
}
```


PDFPageCountError: Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
